In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import random
import json

import warnings
warnings.filterwarnings('ignore')


In [ ]:
import json

# Your JSON string
json_string = '{ "intents": [ { "intent" : "Retrieve_Case", "text": [ "Find cases citing AIR 2012 SC 890 where The appeal is allowed, and the order passed by the High Court is set aside", "Show cases with Company Law", "Retrieve cases citing 2010 SCR 567 with Sales Tax Act where Supreme Court appeals dismissed", "Search cases based on Customs Act citing 1995 SCR 234 by I Verma", "Fetch cases by J Sharma", "Show cases with K Brown, J Agarwal", "Retrieve cases of contract dispute on 30 November, 2025", "Find case Verma vs. Brown and Yadav", "Search cases based on 14 July, 2026 by L Krishnan citing 2005 AIR 1245", "Get cases by S Kapoor of assault" ], "responses" : ["Retrieve_Case"] }, { "intent" : "Retrieve_Statute", "text": [ "Find statutes where Verdict in favor of the plaintiff", "Show statutes citing 1995 SCR 234", "Retrieve statutes where The defendant is found guilty of patent infringement citing 2005 AIR 1245", "Search statutes citing (2002) 3 SCC 567 by L Krishnan", "Fetch statutes involving A Patel", "Show statutes with R Singh, W Agarwal", "Retrieve statutes of contract dispute on 15 June, 2021", "Find statutes of State of Maharashtra vs. Jio Ltd", "Search statutes based on 08 January, 2027 by A Yadav citing 2010 SCR 567", "Get statutes by D Gupta of tax evasion" ], "responses" : ["Retrieve_Statute"] }, { "intent" : "Retrieve_Date", "text": [ "Find date citing 2010 SCR 567 where Supreme Court appeals dismissed", "Show date with IPC Section 302", "Retrieve date citing 1988 AIR 789 and where Verdict in favor of the plaintiff with Evidence Act", "Search date with Income Tax Act citing 1995 SCR 234 by A Patel", "Fetch date by S Kapoor", "Show date with C Sharma, A Yadav, P Krishnan", "Retrieve date of cybercrime where The appeal is allowed, and the order passed by the High Court is set aside", "Find date State of Maharashtra vs. Jio Ltd", "Search date based on 27 October, 2022 by J Sharma citing AIR 2012 SC 890", "Get date by I Verma of robbery" ], "responses" : ["Retrieve_Date"] }, { "intent" : "Retrieve_Author", "text": [ "Find author citing AIR 2000 SC 123 where Verdict in favor of the plaintiff", "Show author of case with Patents Act", "Retrieve author of case following 2005 AIR 1245 and where Tax evasion charges dropped due to lack of evidence with Sales Tax Act", "Search author of case based on Income Tax Act citing (2002) 3 SCC 567 by L Krishnan", "Show author of case with E Krishnan, M Gupta", "Get author with D Gupta, S Sharma with Company Law", "Retrieve author of case of contract dispute on 16 May, 2023", "Find author of case Krishnan & Sons vs. Wilson Enterprises", "Search author based on 15 June, 2021 by M Brown, D Wilson, S Davis citing 2010 SCR 567", "Get author by J Sharma of cybercrime" ], "responses" : ["Retrieve_Author"] }, { "intent" : "Retrieve_Bench", "text": [ "Find bench citing 1988 AIR 789 with Verdict in favor of the plaintiff", "Show bench with CPC Section 210", "Retrieve bench following (2002) 3 SCC 567 and having Tax evasion charges dropped due to lack of evidence with Customs Act", "Search bench based on Patents Act citing 2018 (4) UJ 789 SC by D Gupta", "Fetch bench by R Singh", "Get bench by S Kapoor with Company Law", "Retrieve bench of patent infringement on 31 December, 2023", "Find bench Krishnan & Sons vs. Wilson Enterprises", "Search bench based on 14 July, 2026 by A Patel citing (2014) 6 SCC 321", "Get bench by L Krishnan of cybercrime" ], "responses" : ["Retrieve_Bench"] }, { "intent" : "Retrieve_Verdict", "text": [ "Find verdict citing 2018 (4) UJ 789 SC", "Show verdict with Contract Act", "Retrieve verdict following AIR 2012 SC 890 and having Supreme Court appeals dismissed with Sales Tax Act", "Search verdict based on Customs Act citing 2020 (2) UJ 456 SC by A Yadav", "Fetch verdict by A Patel", "Show verdict with J Patel, A Verma", "Retrieve verdict of patent infringement on 14 July, 2026", "Find verdict Agarwal vs. The Commissioner of Taxation", "Search verdict based on 27 October, 2022 by M Brown, D Wilson, S Davis citing AIR 2000 SC 123", "Get verdict by L Krishnan of tax evasion" ], "responses" : ["Retrieve_Verdict"] }, { "intent" : "Retrieve_Alldeets", "text": [ "Find details for case citing 2018 (4) UJ 789 SC", "Show details for cases with with Contract Act", "Retrieve details following AIR 2012 SC 890 and having Supreme Court appeals dismissed with Sales Tax Act", "Search details based on Customs Act citing 2020 (2) UJ 456 SC by A Yadav", "Fetch details for cases by A Patel", "Show details for cases with J Patel, A Verma", "Retrieve details of patent infringement on 14 July, 2026", "Find details for case with Agarwal vs. The Commissioner of Taxation", "Search details of case on 27 October, 2022 by M Brown, D Wilson, S Davis citing AIR 2000 SC 123", "Get details of case by L Krishnan of tax evasion" ], "responses" : ["Retrieve_Verdict"] } ] }'

# Convert string to JSON (Python dict)
data = json.loads(json_string)

# Now json_object is a Python dictionary
print(data)
print(type(data))

{'intents': [{'intent': 'Retrieve_Case', 'text': ['Find cases citing AIR 2012 SC 890 where The appeal is allowed, and the order passed by the High Court is set aside', 'Show cases with Company Law', 'Retrieve cases citing 2010 SCR 567 with Sales Tax Act where Supreme Court appeals dismissed', 'Search cases based on Customs Act citing 1995 SCR 234 by I Verma', 'Fetch cases by J Sharma', 'Show cases with K Brown, J Agarwal', 'Retrieve cases of contract dispute on 30 November, 2025', 'Find case Verma vs. Brown and Yadav', 'Search cases based on 14 July, 2026 by L Krishnan citing 2005 AIR 1245', 'Get cases by S Kapoor of assault'], 'responses': ['Retrieve_Case']}, {'intent': 'Retrieve_Statute', 'text': ['Find statutes where Verdict in favor of the plaintiff', 'Show statutes citing 1995 SCR 234', 'Retrieve statutes where The defendant is found guilty of patent infringement citing 2005 AIR 1245', 'Search statutes citing (2002) 3 SCC 567 by L Krishnan', 'Fetch statutes involving A Patel', 'Sh

In [ ]:
def clean(line):
    cleaned_line = ''
    for char in line:
        if char.isalpha():
            cleaned_line += char
        else:
            cleaned_line += ' '
    cleaned_line = ' '.join(cleaned_line.split())
    return cleaned_line


In [ ]:
#list of intents
intents = []
unique_intents = []
#all text data to create a corpus
text_input= []
# dictionary mapping intent with appropriate response
response_for_intent = {}
for intent in data['intents']:
    #list of unique intents
    if intent['intent'] not in unique_intents:
        unique_intents.append(intent['intent'])
    for text in intent['text']:
        #cleaning is done before adding text to corpus
        text_input.append(clean(text))
        intents.append(intent['intent'])
    if intent['intent'] not in response_for_intent:
        response_for_intent[intent['intent']] = []
    for response in intent['responses']:
        response_for_intent[intent['intent']].append(response)


In [ ]:
print(len(text_input))
print(len(intents))

70
70


In [ ]:
import random

# Generate a random number between 0 and 160
random_number = random.randint(0, 70)
print(text_input[random_number])
print(intents[random_number])

Get bench by L Krishnan of cybercrime
Retrieve_Bench


In [ ]:
tokenizer = Tokenizer(filters='',oov_token='<unk>')
tokenizer.fit_on_texts(text_input)
sequences = tokenizer.texts_to_sequences(text_input)
padded_sequences = pad_sequences(sequences, padding='pre')
print('Shape of Input Sequence:',padded_sequences.shape)
padded_sequences[:5]


Shape of Input Sequence: (70, 21)


array([[ 8, 14,  5, 15, 16, 27, 12, 74, 39, 75, 29, 12, 76, 77,  3, 12,
        78, 34, 39, 79, 80],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        13, 14,  4, 65, 66],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  9, 14,  5, 30,  4, 49, 17,  7,
        27, 50, 34, 51, 52],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10, 14, 18,  6, 53,  7,
         5, 30,  3, 81, 40],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        35, 14,  3, 36, 41]], dtype=int32)

In [ ]:
intent_to_index = {}
categorical_target = []
index = 0

for intent in intents:
    if intent not in intent_to_index:
        intent_to_index[intent] = index
        index += 1
    categorical_target.append(intent_to_index[intent])

num_classes = len(intent_to_index)
print('Number of Intents :',num_classes)

# Convert intent_to_index to index_to_intent
index_to_intent = {index: intent for intent, index in intent_to_index.items()}
index_to_intent


Number of Intents : 7


{0: 'Retrieve_Case',
 1: 'Retrieve_Statute',
 2: 'Retrieve_Date',
 3: 'Retrieve_Author',
 4: 'Retrieve_Bench',
 5: 'Retrieve_Verdict',
 6: 'Retrieve_Alldeets'}

In [ ]:
categorical_vec = tf.keras.utils.to_categorical(categorical_target,
                                                num_classes=num_classes)

categorical_vec = categorical_vec.astype('int32')

print('Shape of Ca',categorical_vec.shape)
categorical_vec[:5]


Shape of Ca (70, 7)


array([[1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0]], dtype=int32)

In [ ]:
epochs=20
embed_dim=50
lstm_num=364
output_dim=categorical_vec.shape[1]
input_dim=len(unique_intents)
print("Input Dimension :{},\nOutput Dimension :{}".format(input_dim,output_dim))


Input Dimension :7,
Output Dimension :7


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index) + 1, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_num, dropout=0.1)),
    tf.keras.layers.Dense(lstm_num, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(output_dim, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
test_text_inputs = ["Find cases by R Sharma", "Show cases with M Patel", "Retrieve cases on 23 july, 2002 with IPC Section 132", "Search cases with A Verma, M Prasad where sentence is upheld", "Cases by L Yadav", "Find statutes where Appeal dismissed", "Show statutes on 19 June, 2001", "Retrieve statute of case by M Patel", "Search statute by S Patel on 23 march, 2001", "Fetch statutes by S Patel","Find date where Appeal dismissed", "Show date with IPC Section 119", "Retrieve date where The order passed by the High Court is set aside with Income Tax Act", "Search date with Patents Act by M Brown", "Fetch date by J Martin","Show date with B Smith, L WilsoN", "Retrieve date where The appeal is allowed and the order passed by the High Court is set aside", "Find date The People vs. John Smith", "Find author where Appeal dismissed with IPC Section 30", "Show author of case with IPC Section 119", "Retrieve author of case where The order passed by the High Court is set aside with Income Tax Act", "Search author of case based on Patents Act by M Brown", "Show author of case with B Smith, L Wilson", "Retrieve author of case on 25 September, 2026", "Find author of case The People vs. John Smith", "Find bench where Appeal dismissed"  , "Show bench with IPC Section 119", "Retrieve bench where The order passed by the High Court is set aside with Income Tax Act", "Search bench based on Patents Act by M Brown", "Fetch bench by J Martin", "Retrieve bench on 25 September, 2026", "Find bench The People vs. John Smith", "Find verdict by N Modi", "Show verdict with IPC Section 119", "Retrieve verdict following where Appeal dismissed with Income Tax Act", "Search verdict based on Patents Act by M Brown", "Fetch verdict by J Martin", "Show verdict with B Smith, L Wilson", "Retrieve verdict on 25 September, 2026", "Find verdict The People vs. John Smith", "Find details for case citing 2018 (4) UJ 789 SC", "Find details for case with Agarwal vs. The Commissioner of Taxation"]
test_intents = ["Retrieve_Case", "Retrieve_Case","Retrieve_Case","Retrieve_Case","Retrieve_Case", "Retrieve_Statute", "Retrieve_Statute", "Retrieve_Statute", "Retrieve_Statute", "Retrieve_Statute", "Retrieve_Date", "Retrieve_Date", "Retrieve_Date", "Retrieve_Date", "Retrieve_Date", "Retrieve_Date", "Retrieve_Date", "Retrieve_Date", "Retrieve_Author" , "Retrieve_Author", "Retrieve_Author", "Retrieve_Author", "Retrieve_Author", "Retrieve_Author", "Retrieve_Author", "Retrieve_Bench", "Retrieve_Bench", "Retrieve_Bench", "Retrieve_Bench", "Retrieve_Bench", "Retrieve_Bench", "Retrieve_Bench", "Retrieve_Verdict" , "Retrieve_Verdict", "Retrieve_Verdict", "Retrieve_Verdict", "Retrieve_Verdict", "Retrieve_Verdict", "Retrieve_Verdict", "Retrieve_Verdict", "Retrieve_Alldeets","Retrieve_Alldeets"]

print(len(test_text_inputs))
print(len(test_intents))

42
42


In [ ]:

test_sequences = tokenizer.texts_to_sequences(test_text_inputs)
test_padded_sequences = pad_sequences(test_sequences,  padding='pre')
test_labels = np.array([unique_intents.index(intent) for intent in test_intents])
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=num_classes)
loss, accuracy = model.evaluate(test_padded_sequences, test_labels)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.2103 - loss: 1.9453


In [ ]:
from sklearn.metrics import f1_score, recall_score, precision_score
pred_labels = model.predict(test_padded_sequences)
pred_labels = model.predict(test_padded_sequences)
pred_labels = np.argmax(pred_labels, axis=1)

# Convert one-hot encoded labels to integer format
true_labels = np.argmax(test_labels, axis=1)
precision = precision_score(true_labels, pred_labels, average='weighted')
recall = recall_score(true_labels, pred_labels, average='weighted')
f1 = f1_score(true_labels, pred_labels, average='weighted')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step 
Precision: 0.14367413632119513
Recall: 0.19047619047619047
F1 Score: 0.13522126022126021


In [ ]:
def response(sentence):
	sent_tokens = []
	# Split the input sentence into words
	words = sentence.split()
	# Convert words to their corresponding word indices
	for word in words:
		if word in tokenizer.word_index:
			sent_tokens.append(tokenizer.word_index[word])
		else:
			# Handle unknown words
			sent_tokens.append(tokenizer.word_index['<unk>'])
	sent_tokens = tf.expand_dims(sent_tokens, 0)
	#predict numerical category
	pred = model(sent_tokens)
	#category to intent
	pred_class = np.argmax(pred.numpy(), axis=1)
	# random response to that intent
	return random.choice(
		response_for_intent[index_to_intent[pred_class[0]]]), index_to_intent[pred_class[0]]


In [ ]:
print("Note: Enter 'quit' to break the loop.")
while True:
	query = input('You: ')
	if query.lower() == 'quit':
		break
	bot_response, typ = response(query)
	print('Geek: {} -- TYPE: {}'.format(bot_response, typ))
	print()


Note: Enter 'quit' to break the loop.
You: quit
